# SQL CITY Murder Mystery And WEB SCRAPING

In this note book we are going to find the killer using our skill in sql and scrape a book store website (it is at the bottom)

## Setting Up the Connection:

In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
conn = sql.connect("sql-murder-mystery.db")

In [3]:
query = """SELECT name FROM sqlite_master WHERE type='table';"""

pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query,conn)

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


## 1. Retrieve Crime Scene Report

In [23]:
query = """
SELECT *
FROM crime_scene_report
WHERE city = "SQL City" and date = 20180115 and type = 'murder'
"""

pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query,conn)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


## 2. Witness Personal Details

In [24]:
query = """
SELECT *
FROM person 
WHERE address_street_name = 'Northwestern Dr' 
ORDER BY address_number DESC 
LIMIT 1; """

pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [25]:
query = """
SELECT *
FROM person 
WHERE address_street_name = 'Franklin Ave' 
AND name LIKE 'Annabel%';
 """

pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


## 3. View Witness Interviews

In [26]:
query = """
SELECT *
FROM interview 
WHERE person_id IN (14887, 16371);
"""

pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query,conn)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


## 4. Check Gym Database, Car Details, Personal Details and Membership Status 

In [27]:
query = """
SELECT person.name
FROM get_fit_now_member 
JOIN person ON get_fit_now_member.person_id = person.id 
JOIN drivers_license ON person.license_id = drivers_license.id 
WHERE membership_status = 'gold' 
AND get_fit_now_member.id LIKE '48Z%' 
AND plate_number LIKE '%H42W%';
"""

pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query,conn)

,name
0,Jeremy Bowers


In [28]:
query = """
SELECT transcript
FROM interview 
JOIN person ON person.id = interview.person_id 
WHERE name = 'Jeremy Bowers';"""

pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query,conn)

,transcript
0,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [29]:
query = """
SELECT person.name
FROM person
JOIN drivers_license ON person.license_id = drivers_license.id
JOIN facebook_event_checkin ON person.id = facebook_event_checkin.person_id
WHERE drivers_license.height BETWEEN 65 AND 67
AND drivers_license.hair_color = 'red'
AND drivers_license.car_make = 'Tesla'
AND drivers_license.car_model = 'Model S'
AND facebook_event_checkin.event_name = 'SQL Symphony Concert'
GROUP BY person.name
HAVING COUNT(*) = 3;"""

pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query,conn)

,name
0,Miranda Priestly


## 8. Analyze and Draw Conclusions

According to the analysis, we can conclude that Miranda Priestly hired a person to get tha murder done.

## 9. Document Findings

from crime_scene_report we got the details about 2 witnesses with that details we found out the names of the witnesses from person after that we got the transcript of the witnesses from interview and the transcript gave us a few details about the killer with those clues we found out who the killer was and then with the killers transcript we found out who hired him to do the murder.


## 10. Prepare a Report

Witnesses : 1)Morty Schapiro
            2)Annabel Miller

        the killer had a getfit gym bag and the membership number started with 48Z and the car
        plate number included H42W and the killer was seen by the witness on 9th jan
        
        matching all the clue we have one suspect who's name is Jeremy Bowers,
  
Murderer: Jeremy Bowers

        he tells that the person who hire him was a female who had a lot of money, her height
        was 65" or 67", she had red hair and owned a Tesla Model S and she attended the SQL 
        Symphony Concert 3 times in December 2017
        
        matching all the clues we have one suspect who's name is Miranda Priestly,
        
Hirer: Miranda Priestly

# WEB SCRAPING

In [30]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [31]:
page_to_scrape = requests.get("http://books.toscrape.com/")

In [32]:
soup = bs(page_to_scrape.text, "html.parser")

In [34]:
bookshelf = soup.findAll("li", {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"})

In [39]:
title = []
price = []
rating = []
url = []

for book in bookshelf:
    book_title = book.h3.a["title"]
    book_price = book.findAll("p", {"class": "price_color"})[0].text.strip()
    book_rating = book.p["class"][1]
    book_url = book.h3.a["href"]
    full_url = f"http://books.toscrape.com/catalogue/{book_url}"

    title.append(book_title)
    price.append(book_price)
    rating.append(book_rating)
    url.append(full_url)
    
data = {"Title": title,
        "Price": price,
        "Rating": rating,
        "URL": url}

df = pd.DataFrame(data)
df['Price'] = df['Price'].str.replace("Â", "")


In [40]:
df.head()

,Title,Price,Rating,URL
0,A Light in the Attic,£51.77,Three,http://books.toscrape.com/catalogue/catalogue/a-light-in-the-attic_1000/index.html
1,Tipping the Velvet,£53.74,One,http://books.toscrape.com/catalogue/catalogue/tipping-the-velvet_999/index.html
2,Soumission,£50.10,One,http://books.toscrape.com/catalogue/catalogue/soumission_998/index.html
3,Sharp Objects,£47.82,Four,http://books.toscrape.com/catalogue/catalogue/sharp-objects_997/index.html
4,Sapiens: A Brief History of Humankind,£54.23,Five,http://books.toscrape.com/catalogue/catalogue/sapiens-a-brief-history-of-humankind_996/index.html
